# Compression of Differential Expression Data with Deep Autoencoders

**By [Tony Kabilan Okeke](mailto:tko35@drexel.edu)**

## Notebook Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from numpy.random import shuffle

from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import losses
from keras.utils import plot_model
import tensorflow as tf

import matplotlib.pyplot as plt
import multiprocessing as mp
import seaborn as sns
import pandas as pd
import numpy as np
import yaml
import glob
import os

from mlgo import utils

with open("config/config.yaml", "r") as f:
    config = yaml.safe_load(f)

## Load and Preprocess Data

The data used in this notebook was generated by running the `process_batches.sh`
script in the `scripts` directory. The script downloads raw data from DEE2,
and performs Differential Gene Expression analysis on each batch. The results
are combined into a data matrix containing the log2 fold change of each gene
and the enriched GO terms for each dataset processed. These were then split
into training, validation, and test sets. See the README for more details.

In [ ]:
# Load train, test and validation data
with np.load(f"{config['path']['final']}/labels.npz", allow_pickle=True) as D:
    labels = D['labels']
    features = D['features']
    samples = D['samples']

with np.load(f"{config['path']['final']}/train.npz") as D:
    X_train, Y_train = D['X'], D['Y']

with np.load(f"{config['path']['final']}/test.npz") as D:
    X_test, Y_test = D['X'], D['Y']

with np.load(f"{config['path']['final']}/val.npz") as D:
    X_val, Y_val = D['X'], D['Y']

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

# Combine data and drop low variance features
X = np.concatenate([X_train, X_test, X_val])
sel = VarianceThreshold(threshold=1)
sel.fit(X)

# Update training and validation data
X_train = X[:len(X_train), sel.get_support()]
X_test = X[:len(X_test), sel.get_support()]
X_val = X[:len(X_val), sel.get_support()]

# Combine data for visualization
X = np.concatenate([X_train, X_test, X_val])
Y = np.concatenate([Y_train, Y_test, Y_val])

## Data Visualization

Here, I will visualize the data using PCA with different GO terms serving as labels.

In [ ]:
# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

fig, ax = utils.plotpca(X_pca, pca.explained_variance_ratio_, Y[:,0])
ax.set_title(f"PCA for {labels[0]}")
plt.savefig('notebooks/results/pca.png', dpi=300, transparent=False)

In [ ]:
# TSNE
tsne = TSNE(n_components=2, n_jobs=mp.cpu_count(), perplexity=50)
X_tsne = tsne.fit_transform(X)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
g = sns.scatterplot(
    x=X_tsne[:, 0], y=X_tsne[:, 1], hue=Y[:,100], s=100, alpha=0.3, ax=ax
)
g.legend(loc="upper right", fontsize=13, frameon=False)
ax.set_xlabel(f"t-SNE 1", fontsize=13)
ax.set_ylabel(f"t-SNE 2", fontsize=13)
ax.set_title(f"t-SNE for {labels[0]}", fontsize=16)
utils.format(ax)
plt.savefig('notebooks/results/tsne.png', dpi=300, transparent=False)

## Autoencoder Model

Here, I will define the architecture for the autoencoder network I am evaluating.
The function defined below will return a Keras model object with the specified
number of hidden layers and nodes per layer.

### Hyperparameter Optimization

Here, we will evaluate the impact of varying the number of hidden layers and
the total number of hidden units on the performance of the autoencoder.

In [ ]:
# Define parameters
INPUT_DIM = X_train.shape[1]
BATCH_SIZE = 128
EPOCHS = 50

# Define callback for early stopping
callbacks = [
    ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=5, min_delta=0.1, min_lr=0.00001
    )
]


"""
This function will be used to train the model during each iteration of the
hyperparameter optimization.
"""
def evaluate_model():
    # Build the model
    model = utils.build_autoencoder(NUM_LAYERS, NUM_HIDDEN, INPUT_DIM)
    model.compile(optimizer='adam', loss=losses.MeanSquaredError())

    # Train the model
    history = model.fit(
        X_train, 
        X_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        shuffle=True,
        validation_data=(X_val, X_val),
        callbacks=callbacks
    )

    # Store the best validation loss
    best_loss = history.history['loss'][-1]
    best_val_loss = history.history['val_loss'][-1]
    with open('notebooks/results/optimization.tsv', 'a') as f:
        f.write(f'{NUM_LAYERS}\t{NUM_HIDDEN}\t{best_loss}\t{best_val_loss}\n')

#### Varying the Total Number of Hidden Units

Here, we will evaluate the impact of varying the total number of hidden units
with only one hidden layer.

In [ ]:
# Iterate over the number of layers
hidden = [1, 10, 100, 200, 400, 600, 800, 1000, 2000]

# Keep number of layers constant
NUM_LAYERS = 1

for NUM_HIDDEN in hidden:
    # Train the model and store the results
    p = mp.Process(target=evaluate_model)
    p.start()
    p.join()
p.terminate()

In [ ]:
# Load the results
results = pd.read_csv('notebooks/results/optimization.tsv', sep='\t')
results = results[results['num_layers'] == 1]

# Plot the results
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(results['num_hidden'], results['loss'], label='Training')
ax.plot(results['num_hidden'], results['val_loss'], label='Validation')
ax.set_xlabel('Number of Hidden Units', fontsize=13)
ax.set_ylabel('Loss (MSE)', fontsize=13)
ax.legend(fontsize=13, fancybox=False)
utils.format(ax)
plt.tight_layout()
plt.savefig('notebooks/results/optimization-hidden-units.png', dpi=300, transparent=False)

#### Varying the Number of Hidden Layers

Based on the results from the previous experiment, the optimal number of hidden
units is 2000. Here, we will evaluate the impact of varying the number of hidden
layers with a total of 2000 hidden units.

In [ ]:
# Iterate over the number of layers
layers = [2, 3, 4, 5, 6, 7, 8, 9, 10]

# Keep number of hidden units constant
hidden = [1000, 2000]

for NUM_HIDDEN in hidden:
    for NUM_LAYERS in layers:
        # Train the model and store the results
        p = mp.Process(target=evaluate_model)
        p.start()
        p.join()
p.terminate()

In [ ]:
# Load the results
results = pd.read_csv('notebooks/results/optimization.tsv', sep='\t')
results = results[results['num_hidden'] >= 1000]

# Use Dark2 color palette
colors = ['#FF0000', '#000000', '#E7298A', '#66A61E']

# Plot the results (use solid lines for 2000 hidden units and dashed lines for 1000)
fig, ax = plt.subplots(figsize=(6, 4))
for i, (NUM_HIDDEN, ls) in enumerate(zip(hidden[::-1], ['--', '-'])):
    sub = results[results['num_hidden'] == NUM_HIDDEN]
    ax.plot(
        sub['num_layers'], sub['loss'], linestyle=ls, color=colors[i],
        linewidth=2,
        label=f'Training ({NUM_HIDDEN} HU)'
    )
    ax.plot(
        sub['num_layers'], sub['val_loss'], linestyle=ls, color=colors[i+2],
        linewidth=2,
        label=f'Validation ({NUM_HIDDEN} HU)'
    )

ax.set_xlabel('Number of Hidden Layers', fontsize=13)
ax.set_ylabel('Loss (MSE)', fontsize=13)
ax.legend(fontsize=13, loc='upper left')
utils.format(ax)
plt.tight_layout()
plt.savefig('notebooks/results/optimization-hidden-layers.png', dpi=300, transparent=False)

### Model Evaluation with Cross-Validation

Using the best combination of hyperparameters, we will evaluate the performance
of the autoencoder using 4-fold cross-validation.

**Optimal Hyperparameters:**
- `NUM_HIDDEN`: 2000
- `NUM_LAYERS`: 1

In [ ]:
# Clear the results file
with open('notebooks/results/crossval.tsv', 'w') as f:
    pass

In [ ]:
"""
Function modified for Cross-Validation
"""
def evaluate_model():
    # Build the model
    model = utils.build_autoencoder(NUM_LAYERS, NUM_HIDDEN, INPUT_DIM)
    model.compile(optimizer='adam', loss=losses.MeanSquaredError())

    # Train the model
    history = model.fit(
        X_fold_train, 
        X_fold_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        shuffle=True,
        validation_data=(X_fold_val, X_fold_val),
        callbacks=callbacks
    )

    # Store the best validation loss
    best_val_loss = history.history['val_loss'][-1]
    with open('notebooks/results/crossval.tsv', 'a') as f:
        f.write(f'{best_val_loss}\n')

In [ ]:
# Hyperparameters for Best Model
NUM_LAYERS = 1
NUM_HIDDEN = 2000

# Define folds for cross-validation
NFOLDS = 4

# Split training data
shuffle(X_train)
shuffle(X_val)
training = np.array_split(X_train, NFOLDS)
validation = np.array_split(X_val, NFOLDS)

# Cross-validation
val_loss = []
for X_fold_train, X_fold_val in zip(training, validation):
    # Train the model and store the results
    p = mp.Process(target=evaluate_model)
    p.start()
    p.join()

# Report the average validation loss
results = pd.read_csv('notebooks/results/crossval.tsv', sep='\t', header=None)
print(f"Cross-Validation Loss: {results.mean().values[0]:.4f}")

In [ ]:
# Visualize the model architecture
model = utils.build_autoencoder(NUM_LAYERS, NUM_HIDDEN, INPUT_DIM)
model.compile(optimizer='adam', loss=losses.MeanSquaredError())
plot_model(model, to_file='notebooks/results/model.png', show_shapes=True, show_layer_names=True)